In [51]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

In [54]:
# path = osp.join(osp.dirname(osp.realpath(__file__)), '', 'data', 'Planetoid')
dataset = Planetoid(root='cora_dataset_n', name='Cora')
data=dataset[0]
print('Test data:',dataset[0])
# dataset = Planetoid(path, name='Cora',transform=transform )
# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.
train_data, val_data, test_data = dataset[0]

Test data: Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


Processing...
Done!


ValueError: too many values to unpack (expected 3)

In [55]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


In [56]:
model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [57]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss



In [58]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


In [59]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Epoch: 001, Loss: 0.6930, Val: 0.6669, Test: 0.6988
Epoch: 002, Loss: 0.6825, Val: 0.6648, Test: 0.6922
Epoch: 003, Loss: 0.7094, Val: 0.6702, Test: 0.6999
Epoch: 004, Loss: 0.6774, Val: 0.6938, Test: 0.7253
Epoch: 005, Loss: 0.6851, Val: 0.7223, Test: 0.7549
Epoch: 006, Loss: 0.6884, Val: 0.7192, Test: 0.7593
Epoch: 007, Loss: 0.6890, Val: 0.6904, Test: 0.7367
Epoch: 008, Loss: 0.6877, Val: 0.6699, Test: 0.7167
Epoch: 009, Loss: 0.6845, Val: 0.6613, Test: 0.7043
Epoch: 010, Loss: 0.6789, Val: 0.6586, Test: 0.6962
Epoch: 011, Loss: 0.6755, Val: 0.6622, Test: 0.6918
Epoch: 012, Loss: 0.6778, Val: 0.6681, Test: 0.6902
Epoch: 013, Loss: 0.6732, Val: 0.6756, Test: 0.6888
Epoch: 014, Loss: 0.6676, Val: 0.6831, Test: 0.6908
Epoch: 015, Loss: 0.6642, Val: 0.6918, Test: 0.6954
Epoch: 016, Loss: 0.6602, Val: 0.6975, Test: 0.6954
Epoch: 017, Loss: 0.6548, Val: 0.6986, Test: 0.6921
Epoch: 018, Loss: 0.6459, Val: 0.7006, Test: 0.6912
Epoch: 019, Loss: 0.6417, Val: 0.7117, Test: 0.7037
Epoch: 020, 

In [64]:
train_data.x, train_data.edge_index

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[1358,  505,    6,  ..., 2274, 2472, 1681],
         [1731, 1448, 1042,  ..., 1370,  192,  459]]))

In [60]:
# saving complete model
torch.save(model, 'Link_prediction_GNN_model_weights.pth')

In [61]:
# load model
model = torch.load('Link_prediction_GNN_model_weights.pth')

In [67]:
model(test_data)

NotImplementedError: 

In [50]:
len(data)

3

In [45]:
y_pred=model(data[1])
y_pred

NotImplementedError: 